### PA1-1&2: Scheduling

In [18]:
import heapq

def heap_sort(arr):
    # using heapify to convert list into heap 
    heapq.heapify(arr) 
    # sort array usign extract-min operation
    return [heapq.heappop(arr) for i in range(len(arr))]

def make_schedule(scores):
    # sort scores by decreasing order into a dictionary to narrow down duplicates 
    heapq.heapify(scores)   
    d_scores = {}   
    while len(scores)>0:
        score = heapq.heappop(scores)
        if -score[0] not in d_scores:
            d_scores[-score[0]] = []
        d_scores[-score[0]].append(score[1:])
    # make schedule: if duplicated scores exist sort by decreasing job weight
    schedule = []
    for k,j in d_scores.items():
        if len(j)>1:
            j = heap_sort([(-1*w,l) for w,l in j])
            j = [(-1*w,l) for w,l in j]
            d_scores[k] = j
        schedule.extend(j)
    return schedule

def cost(jobs):
    C,wC = 0,0
    for w,l in jobs:
        C += l
        wC += C*w
    return wC

# Test case
# jobs = [(8,50),(74,59),(31,73),(45,79),(24,10),(41,66),(93,43),(88,4),(28,30),(41,13),(4,70),(10,58)]
# Assignment data
jobs = []
with open('Jobs.txt') as f:
    for i,l in enumerate(f):
        if i!=0:
            jobs.append(tuple(map(int,l.strip().split())))

# 1. COMPUTE SCORES
# scores = [(-1*(w-l),w,l) for (w,l) in jobs]
scores = [(-1*(w/l),w,l) for (w,l) in jobs]
# 1. OPTIMAL SCHEDULE
schedule = make_schedule(scores) 
# 2. WEIGHTED COST
print('weighted cost = ',cost(schedule))

weighted cost =  67311454237


### PA1-3: Prim's MST

In [8]:
# adj = '''
# 1 2 6
# 1 4 5
# 1 5 4
# 2 4 1
# 2 5 2
# 2 3 5 
# 2 6 3
# 3 6 4
# 4 5 2
# 5 6 4
# '''

# g = {}
# for i,l in enumerate(adj.split('\n')):
#     if (len(l)>0) and i!=0:
#         v1,v2,c = tuple(map(int,l.split()))
#         if v1 not in g:
#             g[v1] = set()
#         g[v1].add((v2,c))
#         if v2 not in g:
#             g[v2] = set()
#         g[v2].add((v1,c))

In [19]:
g = {}
with open('Edges.txt') as f:
    for i,l in enumerate(f):
        if i!=0:
            v1,v2,c = tuple(map(int,l.split()))
            if v1 not in g:
                g[v1] = set()
            g[v1].add((v2,c))
            if v2 not in g:
                g[v2] = set()
            g[v2].add((v1,c))

In [23]:
V = set(g.keys())
X = set()
T = []
A = [10000 for i in range(len(V)+1)]

# initialize source node
s = 1
X.add(s)

# initialize heap := (cost, node1, node2)
h = []
d_h = {}
i=0
for v in V:
    if v!=s:
        d_h[v] = i
        h.append((10000,10000,v)) # assign 10000 to initial values of cost,node1
        i+=1
# assign keys to heap edges starting in source node
for w,c in g[s]:
    i = d_h[w]
    h[i] = (c,s,w)

heapq.heapify(h)
# print(h)

In [21]:
# heap implementation
# Disclaimer: heap deletion is done in log(n) time, which kills benefit of using heap to select optimal node
while h:

    c, v1, v2 = heapq.heappop(h)
    T.append(((v1,v2),c))
#     print('extracted', c,v1,v2)
    X.add(v2)

    for w,c_w in g[v2]:
        if w not in X:
#             print(v2,w)
            # delete w from heap: log(n). Is it possible store ix in d_h?
            for i,(c,w1,w2) in enumerate(h):
                if w==w2:
                    ix=i
            h[ix] = h[-1]
            h.pop()
            heapq.heapify(h)
            # recompute key
            new_c = min(A[w],c_w)
            # re-insert into heap
            heapq.heappush(h,(new_c,v2,w))
            A[w] = new_c
#     print(h)

In [22]:
mst = 0
for e,c in T:
    mst += c
print(mst)

-3612829
